<a href="https://colab.research.google.com/github/AlewaIvanovi4/ml_practise/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **firts example**

In [19]:
# Загружаем библиотеки
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler

import pandas as pd
import numpy as np

In [5]:
# Загружаем данные из встроенного набора библиотеки
data = load_iris()
X = data.data # Признаки
y = data.target # Целевая переменная

In [7]:
# Делим данные на тренировочную и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Инициализация модели
model = LogisticRegression()

# Обучение модели
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Оценка точности модели
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy * 100:.2f}%")

Точность модели: 100.00%


In [11]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/orders_seafood.csv')

In [14]:
df.head()

,client_id,reject_count,confirm_count,last_summ,summ_,count_position,target
0,8195,27,13,132684.4,34875.3,18,1
1,16392,29,15,149790.1,19537.4,11,1
2,16396,25,17,80195.2,44583.0,23,1
3,8206,22,10,80089.5,22561.6,15,1
4,16398,14,14,163565.6,27986.4,17,1


In [15]:
df.isna().mean()

,0
client_id,0.0
reject_count,0.0
confirm_count,0.0
last_summ,0.0
summ_,0.0
count_position,0.0
target,0.0


In [17]:
RANDOM_STATE = 77

X = df.drop(columns=['target', 'client_id'])
y = df['target']

In [24]:
scaler = StandardScaler()

X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)
X_train_scalled = scaler.fit_transform(X_train)
X_test_scalled = scaler.transform(X_test)

clf = LogisticRegression()
clf = clf.fit(X_train_scalled, y_train)
y_pred = clf.predict(X_test_scalled)

In [25]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy * 100:.2f}%")

Точность модели: 64.89%


Понять, достаточно ли точна модель, можно, только оценив её влияние на показатели бизнеса. Напомним, наша цель — сэкономить с помощью модели 1 млн 100 тыс. рублей. Для проверки предлагаем сделать приблизительный расчёт на тестовой выборке:

* Вычислить, сколько компании удалось бы сберечь благодаря модели. Для этого нужно умножить стоимость отменённых заказов, которые удалось предсказать, на 70% — размер предоплаты. Но важно учесть, что 20% клиентов из-за нового требования изначально не оформили бы заказ — соответственно, предоплату не внесли бы. Поэтому рассчитанную сумму выгоды нужно умножить ещё и на 80%, чтобы её уточнить. То есть слегка уменьшить.
* Найти сумму, которую компания потеряла бы из-за модели. Чтобы её выяснить, тоже важна статистика об уходе пятой части клиентов из-за запроса на предоплату. Надо умножить на 20% стоимость заказов, отмена которых не подтвердилась. Ведь их могли бы оплатить, если бы никого не смутило новое требование.
* Вычесть из сбережённой суммы объём потерь.

In [44]:
predicts = X_test[['summ_']].copy()
predicts['y_pred'] = y_pred
predicts['y_test'] = y_test

TP = predicts[(predicts['y_pred']==1)&(predicts['y_test']==1)]['summ_'].sum()*0.7*0.8
FP = predicts[(predicts['y_pred']==1)&(predicts['y_test']==0)]['summ_'].sum()*0.2

money = TP-FP
formatted = f"{money:_.2f}".replace("_", " ")

print(formatted)

1 066 557.42


In [51]:
trshlds = []
mny = []

for i in thresholds:
    y_pred_log = []
    for j in y_proba_log:
        y_pred_log.append(1 if j >= i else 0)

    predicts = pd.DataFrame(
        zip(y_pred_log, y_test, X_test['summ_'].tolist()),
        columns=['y_pred', 'y_test', 'summ_']
    )

    TP = predicts[(predicts['y_pred'] == 1) & (predicts['y_test'] == 1)]['summ_'].sum() * 0.7 * 0.8
    FP = predicts[(predicts['y_pred'] == 1) & (predicts['y_test'] == 0)]['summ_'].sum() * 0.2
    money = TP - FP
    formatted = f"{money:_.2f}".replace("_", " ")

    trshlds.append(i)
    mny.append(formatted)

# Построение финального DataFrame
result_df = pd.DataFrame(data={
    'threshold': trshlds,
    'money': mny
})

In [52]:
result_df

,threshold,money
0,0.10,1 089 660.98
1,0.18,1 089 660.98
2,0.25,1 089 660.98
3,0.32,1 089 660.98
4,0.40,1 087 737.37
5,0.48,1 080 149.35
6,0.55,1 049 577.92
7,0.62,1 022 793.44
8,0.70,982 374.76
9,0.77,941 061.45
